In [34]:
import pandas as pd
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
import datetime as dt
import matplotlib.dates as md

In [37]:
df = pd.read_parquet('escooter_history.parquet', engine='pyarrow')
df['weekday'] = pd.Series(df.datetime).dt.day_name()
df.workingday = 1
df.holiday = 0
# set Saturdays and Sundays not as Workingday
df.loc[df['weekday'].isin(['Saturday','Sunday']), 'workingday'] = 0

# set Tuesdays aans Wednesday as Workingday, but not as Holiday
df.loc[df['weekday'].isin(['Tuesday','Wednesday']), 'workingday'] = 1
df.loc[df['weekday'].isin(['Tuesday','Wednesday']), 'holiday'] = 0

# List of Holidays
# Based on https://www.officeholidays.com/countries/usa/2019 , https://www.officeholidays.com/countries/usa/2020
holidays2019 = [(2019,1,1),(2019,1,21),(2019,2,18),(2019,5,27),(2019,7,4),(2019,9,2),(2019,10,14),(2019,11,11),(2019,11,28),(2019,12,25)]
holidays2020 = [(2020,1,1),(2020,1,20),(2020,2,17),(2020,5,25),(2020,7,3),(2020,7,4),(2020,9,7),(2020,10,12),(2020,11,11),(2020,11,26),(2020,12,25)]

# function to set Holidays to 1, Working Day to 0.
def setHolidays (t):
    y = t[0]
    m = t[1]
    d = t[2]
    
    start = dt.datetime.strptime(f'{y}-{m}-{d} 00:00:00', '%Y-%m-%d %H:%M:%S')
    end = dt.datetime.strptime(f'{y}-{m}-{d+1} 00:00:00', '%Y-%m-%d %H:%M:%S')
 
    df.loc[(df.datetime >= start) & (df.datetime < end), 'holiday'] = 1
    df.loc[(df.datetime >= start) & (df.datetime < end), 'workingday'] = 0
for h in (holidays2019+holidays2020): 
    setHolidays(h) 

In [38]:
df['rc_true'] = df['registered_customer']
df["rc_true"] = df["rc_true"].astype(int)
df["rc_false"] = 1 - df["rc_true"]

df6 = df.groupby([df.datetime.dt.to_period('H')]).agg({
    'holiday':'max',
    'workingday':'max',
    'weather' : 'max',
    'temp' : 'max',
    'atemp' : 'max',
    'humidity': 'max',
    'windspeed' : 'max',
    'rc_true': 'sum',
    'rc_false': 'sum'
            })

df6.to_csv('df_reduceV8.csv')

In [39]:
df6.reset_index(level=0, inplace=True)
df6['datetime'] = df6['datetime'].astype('string') 
df6['datetime'] = df6['datetime'].astype('datetime64') 
df6.to_parquet('newEScooter2.parquet')

In [40]:
df6.dtypes

datetime      datetime64[ns]
holiday                int64
workingday             int64
weather               object
temp                 float64
atemp                float64
humidity               int64
windspeed            float64
rc_true                int32
rc_false               int32
dtype: object

In [41]:
df = pd.read_parquet('newEScooter2.parquet', engine='pyarrow')
df

,datetime,holiday,workingday,weather,temp,atemp,humidity,windspeed,rc_true,rc_false
0,2019-01-01 00:00:00,1,0,"clear, few clouds",9.84,14.395,81,0.0000,13,3
1,2019-01-01 01:00:00,1,0,"clear, few clouds",9.02,13.635,80,0.0000,32,8
2,2019-01-01 02:00:00,1,0,"clear, few clouds",9.02,13.635,80,0.0000,27,5
3,2019-01-01 03:00:00,1,0,"clear, few clouds",9.84,14.395,75,0.0000,10,3
4,2019-01-01 04:00:00,1,0,"clear, few clouds",9.84,14.395,75,0.0000,1,0
...,...,...,...,...,...,...,...,...,...,...
17374,2020-12-31 19:00:00,0,1,"cloudy, mist",10.66,12.880,60,11.0014,299,7
17375,2020-12-31 20:00:00,0,1,"cloudy, mist",10.66,12.880,60,11.0014,199,5
17376,2020-12-31 21:00:00,0,1,"clear, few clouds",10.66,12.880,60,11.0014,146,3
17377,2020-12-31 22:00:00,0,1,"clear, few clouds",10.66,13.635,56,8.9981,109,5


In [42]:
df = df.groupby([df.datetime.dt.to_period('D')]).agg({
    'holiday' : 'max',
    'workingday' : 'max',
    'rc_true': 'sum',
    'rc_false': 'sum'
            })
df

,holiday,workingday,rc_true,rc_false
datetime,,,,
2019-01-01,1,0,654,331
2019-01-02,0,1,670,131
2019-01-03,0,1,1229,120
2019-01-04,0,1,1454,108
2019-01-05,0,0,1518,82
...,...,...,...,...
2020-12-27,0,0,4024,207
2020-12-28,0,1,4381,248
2020-12-29,0,1,2777,310


In [54]:
df.describe().round(2)

,holiday,workingday,rc_true,rc_false
count,731.00,731.00,731.00,731.00
mean,0.03,0.69,3665.08,843.84
std,0.17,0.46,1492.75,676.94
min,0.00,0.00,17.00,5.00
25%,0.00,0.00,2631.00,323.00
50%,0.00,1.00,3633.00,706.00
75%,0.00,1.00,4739.00,1111.50
max,1.00,1.00,6959.00,3410.00


In [55]:
df.workingday.max()

1

In [56]:
pd.crosstab(df['holiday'],df['workingday'])

workingday,0,1
holiday,,
0,207,503
1,21,0


In [57]:
df.loc[df['holiday'] == 1].rc_true.sum() / 21

3686.0

In [58]:
df.loc[df['holiday'] == 0].rc_true.sum() / (731-21)

3664.461971830986

In [59]:
df.loc[df['workingday'] == 1].rc_true.sum() / 503

3520.2564612326046

In [60]:
df.loc[df['workingday'] == 0].rc_true.sum() / (207+21)

3984.5833333333335

In [61]:
df.loc[(df['workingday'] == 0) & (df['holiday'] == 0)].rc_true.sum() / 207

4014.8743961352657

In [62]:
df.loc[df['holiday'] == 1].rc_false.sum() / 21

916.2380952380952

In [63]:
df.loc[df['holiday'] == 0].rc_false.sum() / (731-21)

841.6985915492958

In [64]:
df.loc[df['workingday'] == 1].rc_false.sum() / 503

959.817097415507

In [65]:
df.loc[df['workingday'] == 0].rc_false.sum() / (207+21)

587.9780701754386

In [66]:
df.loc[(df['workingday'] == 0) & (df['holiday'] == 0)].rc_false.sum() / 207

554.6763285024155

In [69]:
df['rc_sum'] = df.rc_true + df.rc_false

In [70]:
df

,holiday,workingday,rc_true,rc_false,rc_sum
datetime,,,,,
2019-01-01,1,0,654,331,985
2019-01-02,0,1,670,131,801
2019-01-03,0,1,1229,120,1349
2019-01-04,0,1,1454,108,1562
2019-01-05,0,0,1518,82,1600
...,...,...,...,...,...
2020-12-27,0,0,4024,207,4231
2020-12-28,0,1,4381,248,4629
2020-12-29,0,1,2777,310,3087


In [71]:
df.loc[df['holiday'] == 1].rc_sum.sum() / 21

4602.238095238095

In [72]:
df.loc[df['holiday'] == 0].rc_sum.sum() / (731-21)

4506.160563380282

In [73]:
df.loc[df['workingday'] == 1].rc_sum.sum() / 503

4480.073558648111

In [74]:
df.loc[df['workingday'] == 0].rc_sum.sum() / (207+21)

4572.561403508772

In [75]:
df.loc[(df['workingday'] == 0) & (df['holiday'] == 0)].rc_sum.sum() / 207

4569.550724637681